# Fraud Detection in Electricity and Gas Consumption

The Tunisian Company of Electricity and Gas (STEG) is a public and a non-administrative company, it is responsible for delivering electricity and gas across Tunisia. The company suffered tremendous losses in the order of 200 million Tunisian Dinars due to fraudulent manipulations of meters by consumers.

Using the client’s billing history, the aim of the challenge is to detect and recognize clients involved in fraudulent activities.

The solution will enhance the company’s revenues and reduce the losses caused by such fraudulent activities.

# The Dataset

The following files are provided:

    - Client_train.csv : Client information in the train population
    - Invoice_train.csv: Clients invoice in the train set
    - Client_test.csv  : Client information for the test population
    - Invoice_test.csv : Clients invoice in the test set

As well as a sample output file:

    - SampleSubmission.csv - is an example of what your submission file should look like. The order of the rows does not matter, but the names of the IDs must be correct. The column "target" is your prediction.

<!-- # # Marking Criteria
# 
# There are multiple ways one can tackle the problem. It is up to you to decide and motivate your decisions. You are not limited to using any specific methods or models.
# 
# Motivate all of your steps taken in completing the task at hand, this includes but is not limited to:
# - Exploring the data
# - Data augmentation
# - Preprocessing the data
# - Choosing the right machine learning model(s)
# - Tuning the model(s)
# - Performance assessment of the model(s) -->

# Variable definitions

### Client data:

    - Client_id: Unique id for client
    - District: District where the client is
    - Client_catg: Category client belongs to
    - Region: Area where the client is
    - Creation_date: Date client joined
    - Target: fraud:1 , not fraud: 0

### Invoice data

    - Client_id: Unique id for the client
    - Invoice_date: Date of the invoice
    - Tarif_type: Type of tax
    - Counter_number:
    - Counter_statue: takes up to 5 values such as working fine, not working, on hold statue, ect
    - Counter_code:
    - Reading_remarque: notes that the STEG agent takes during his visit to the client (e.g: If the counter shows something wrong, the agent gives a bad score)
    - Counter_coefficient: An additional coefficient to be added when standard consumption is exceeded
    - Consommation_level_1: Consumption_level_1
    - Consommation_level_2: Consumption_level_2
    - Consommation_level_3: Consumption_level_3
    - Consommation_level_4: Consumption_level_4
    - Old_index: Old index
    - New_index: New index
    - Months_number: Month number
    - Counter_type: Type of counter

# Marking Criteria

There are multiple ways one can tackle the problem. It is up to you to decide and motivate your decisions. You are not limited to using any specific methods or models.

Motivate all of your steps taken in completing the task at hand, this includes but is not limited to:
- Exploring the data
- Data augmentation
- Preprocessing the data
- Choosing the right machine learning model(s)
- Tuning the model(s)

- Performance assessment of the model(s)

Make sure to add your thoughts and motivations behind your decision making as well as the outcome thereof in markdown cells.

**Your commentary will be more valuable than the performance of the model itself.**

Remeber to label all graphs and keep your notebook neat.

# Make sure to include an Accuracy score of you final model on the test set.
You may choose any other additional criteria you want to evaluate you final model on.

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from collections import Counter

sns.set()

%matplotlib inline
client_data = pd.read_csv('/Users/torick/Documents/Bootcamp/Final Project-Fraud Detection/archive/client_train.csv')
invoice_data = pd.read_csv('/Users/torick/Documents/Bootcamp/Final Project-Fraud Detection/archive/invoice_train.csv')

#Changing the date columns to datetime
client_data['creation_date']= pd.to_datetime(client_data['creation_date'])
invoice_data['invoice_date']= pd.to_datetime(invoice_data['invoice_date'])

/Users/torick/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
client_data.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,1994-12-31,0.0
1,69,train_Client_1,11,107,2002-05-29,0.0
2,62,train_Client_10,11,301,1986-03-13,0.0
3,69,train_Client_100,11,105,1996-11-07,0.0
4,62,train_Client_1000,11,303,2014-10-14,0.0


In [3]:
#Calculate how many invoices per client
client_data['invoice_amount'] = invoice_data.groupby('client_id').size().values
client_data.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_amount
0,60,train_Client_0,11,101,1994-12-31,0.0,35
1,69,train_Client_1,11,107,2002-05-29,0.0,37
2,62,train_Client_10,11,301,1986-03-13,0.0,18
3,69,train_Client_100,11,105,1996-11-07,0.0,20
4,62,train_Client_1000,11,303,2014-10-14,0.0,14


In [4]:
#Calculate how long the client has had a contract with the comapany
today = pd.to_datetime("today")
#print(today.year)

client_data['contract_time_length'] = today.year - client_data['creation_date'].dt.year
client_data.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_amount,contract_time_length
0,60,train_Client_0,11,101,1994-12-31,0.0,35,27
1,69,train_Client_1,11,107,2002-05-29,0.0,37,19
2,62,train_Client_10,11,301,1986-03-13,0.0,18,35
3,69,train_Client_100,11,105,1996-11-07,0.0,20,25
4,62,train_Client_1000,11,303,2014-10-14,0.0,14,7


In [7]:

client_data['contract_time_to_latest'] = invoice_data.groupby('client_id')['invoice_date'].max().dt.year.values- client_data['creation_date'].dt.year.values
    
client_data.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_amount,contract_time_length,contract_time_to_latest
0,60,train_Client_0,11,101,1994-12-31,0.0,35,27,25
1,69,train_Client_1,11,107,2002-05-29,0.0,37,19,17
2,62,train_Client_10,11,301,1986-03-13,0.0,18,35,33
3,69,train_Client_100,11,105,1996-11-07,0.0,20,25,16
4,62,train_Client_1000,11,303,2014-10-14,0.0,14,7,5


In [8]:
#Now having a look at contract creation to latest invoice date

client_data['contract_time_to_latest'] = invoice_data.groupby('client_id')['invoice_date'].max().dt.year.values- client_data['creation_date'].dt.year.values
    
client_data.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_amount,contract_time_length,contract_time_to_latest
0,60,train_Client_0,11,101,1994-12-31,0.0,35,27,25
1,69,train_Client_1,11,107,2002-05-29,0.0,37,19,17
2,62,train_Client_10,11,301,1986-03-13,0.0,18,35,33
3,69,train_Client_100,11,105,1996-11-07,0.0,20,25,16
4,62,train_Client_1000,11,303,2014-10-14,0.0,14,7,5


In [9]:
invoice_data.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,index_dif
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC,82
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC,1384
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC,123
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC,102
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC,572


In [10]:
#Old index versus new index ... no info on this , ploting the difference between new and old
#Create new column
invoice_data['index_dif'] = np.abs(invoice_data['new_index']-invoice_data['old_index'])


In [11]:
#Drop columns not deemed neccesary
reduced_invoice_data = invoice_data.drop(columns=['counter_code','counter_number','old_index','new_index'])

In [13]:
#Hot encode the categorical variables
cat_cols = ['tarif_type','counter_statue','reading_remarque','counter_coefficient','counter_type']

X = pd.get_dummies(reduced_invoice_data,columns=cat_cols)

In [14]:
X.head()

,client_id,invoice_date,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,months_number,index_dif,tarif_type_8,tarif_type_9,...,counter_coefficient_9,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_type_ELEC,counter_type_GAZ
0,train_Client_0,2014-03-24,82,0,0,0,4,82,0,0,...,0,0,0,0,0,0,0,0,1,0
1,train_Client_0,2013-03-29,1200,184,0,0,4,1384,0,0,...,0,0,0,0,0,0,0,0,1,0
2,train_Client_0,2015-03-23,123,0,0,0,4,123,0,0,...,0,0,0,0,0,0,0,0,1,0
3,train_Client_0,2015-07-13,102,0,0,0,4,102,0,0,...,0,0,0,0,0,0,0,0,1,0
4,train_Client_0,2016-11-17,572,0,0,0,12,572,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
df_mean = X.groupby('client_id').agg('mean').iloc[:,6:]
df_mean.head(30)

,tarif_type_8,tarif_type_9,tarif_type_10,tarif_type_11,tarif_type_12,tarif_type_13,tarif_type_14,tarif_type_15,tarif_type_18,tarif_type_21,...,counter_coefficient_9,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_type_ELEC,counter_type_GAZ
client_id,,,,,,,,,,,,,,,,,,,,,
train_Client_0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
train_Client_1,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
train_Client_10,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
train_Client_100,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
train_Client_1000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
train_Client_10000,0.0,0.0,0.0,0.604167,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604167,0.395833
train_Client_100000,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.500000
train_Client_100001,0.0,0.0,0.0,0.512821,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.512821,0.487179
train_Client_100002,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000


In [16]:
df_cons= X.groupby('client_id').agg({'consommation_level_1': ['min', 'max','mean','std'],
                                     'consommation_level_2': ['min', 'max','mean','std'],
                                     'consommation_level_3': ['min', 'max','mean','std'],
                                     'consommation_level_4': ['min', 'max','mean','std'],
                                     'index_dif': ['min', 'max','mean','std'],
                                     'months_number': ['min', 'max','mean','std']})

df_cons.columns = ['consommation_level_1_min','consommation_level_1_max','consommation_level_1_mean','consommation_level_1_std','consommation_level_2_min','consommation_level_2_max','consommation_level_2_mean','consommation_level_2_std','consommation_level_3_min','consommation_level_3_max','consommation_level_3_mean','consommation_level_3_std','consommation_level_4_min','consommation_level_4_max','consommation_level_4_mean','consommation_level_4_std','index_dif_min','index_dif_max','index_dif_mean','index_dif_std','months_number_min','months_number_max','months_number_mean','months_number_std']
df_cons.head()

,consommation_level_1_min,consommation_level_1_max,consommation_level_1_mean,consommation_level_1_std,consommation_level_2_min,consommation_level_2_max,consommation_level_2_mean,consommation_level_2_std,consommation_level_3_min,consommation_level_3_max,...,consommation_level_4_mean,consommation_level_4_std,index_dif_min,index_dif_max,index_dif_mean,index_dif_std,months_number_min,months_number_max,months_number_mean,months_number_std
client_id,,,,,,,,,,,,,,,,,,,,,
train_Client_0,38,1200,352.400000,310.343472,0,186,10.571429,43.568935,0,0,...,0.000000,0.000000,38,1386,362.971429,341.553930,2,12,4.628571,2.101620
train_Client_1,190,1207,557.540541,197.935960,0,0,0.000000,0.000000,0,0,...,0.000000,0.000000,190,1207,557.540541,197.935960,2,8,4.324324,1.375461
train_Client_10,188,2400,798.611111,513.841374,0,682,37.888889,160.748942,0,0,...,0.000000,0.000000,188,3082,836.500000,646.808386,4,12,6.444444,3.399346
train_Client_100,0,15,1.200000,3.607011,0,0,0.000000,0.000000,0,0,...,0.000000,0.000000,0,15,1.200000,3.607011,4,6,4.200000,0.615587
train_Client_1000,124,800,663.714286,224.831365,0,400,104.857143,167.155320,0,800,...,36.714286,105.421081,124,2382,922.642857,633.485669,2,4,3.714286,0.726273


In [17]:
#merging the grouped invoice data set of means and describe()
derived_invoice = df_cons.merge(df_mean,on='client_id')

In [18]:
#merging grouped invoice dataset with the original client dataset
final_dataset = client_data.merge(derived_invoice,on='client_id')

In [20]:
final_dataset.fillna(0,inplace=True) #There are NaN where because standard deviation of one invoice gives error
final_dataset.head(20) 

,disrict,client_id,client_catg,region,creation_date,target,invoice_amount,contract_time_length,contract_time_to_latest,consommation_level_1_min,...,counter_coefficient_9,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_type_ELEC,counter_type_GAZ
0,60,train_Client_0,11,101,1994-12-31,0.0,35,27,25,38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
1,69,train_Client_1,11,107,2002-05-29,0.0,37,19,17,190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
2,62,train_Client_10,11,301,1986-03-13,0.0,18,35,33,188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
3,69,train_Client_100,11,105,1996-11-07,0.0,20,25,16,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
4,62,train_Client_1000,11,303,2014-10-14,0.0,14,7,5,124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
5,69,train_Client_10000,11,103,1993-09-29,0.0,48,28,26,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604167,0.395833
6,62,train_Client_100000,11,309,2012-07-06,0.0,40,9,7,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.500000
7,60,train_Client_100001,11,101,2006-12-04,0.0,78,15,13,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.512821,0.487179
8,62,train_Client_100002,11,301,2006-12-31,0.0,2,15,1,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
9,60,train_Client_100003,11,101,2011-10-25,0.0,5,10,3,254,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
